# Download dataset *

In [ ]:
!wget https://bashupload.com/-nc5I/train2014.zip
!unzip train2014.zip
!rm train2014.zip
!wget https://bashupload.com/baIo3/val2014.zip
!unzip val2014.zip
!rm val2014.zip
!wget http://cs.stanford.edu/people/karpathy/deepimagesent/caption_datasets.zip
!unzip caption_datasets.zip
!rm caption_datasets.zip
!rm dataset_flickr30k.json
!rm dataset_flickr8k.json

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#Data preprocessing *

In [ ]:
import os
import numpy as np
import h5py
import json
import torch
from PIL import Image
from tqdm import tqdm
from collections import Counter
from random import seed, choice, sample
import cv2

In [ ]:
# Datasets:
#   train2014 is a folder of image files for training
#   val2014 is a folder of image files for validation
#   dataset_coco.json is a JSON file that tells you {image -> captions}

## Data loading *


In [ ]:
# Load JSON file into dict
json_path = 'dataset_coco.json'
with open(json_path, 'r') as j:
    data = json.load(j)
print(data['images'][0])

# Understand how each image is captioned
# 'filename' is the image name
# 'filepath' is the folder name
# 'imgid' is the id of the image
# 'sentences' is a list of the human captioning
# 'tokens' is a list of words

In [ ]:
data["images"]

In [ ]:
# Each image may have multiple captions
# to reduce the bias we are introducing, 
# let's use the same number of captions per image
captions_per_image=5

# Maximum number of words in a sentence
# If the sentence has more than max_len words, we skip it
# If the sentence has less than max_len words, we pad it with <pad>
max_len=50

# From json object to a list of (image_path, captions) pairs 
# note: captions should be a list of word lists
train_img_cap_pairs = []
val_img_cap_pairs = []
# test_img_cap_pairs = []

# It contains all distinct words
word_set = set()

for img_obj in data['images']:
    captions = []
    for caption in img_obj['sentences']:
        word_set.update(caption['tokens'])
        if len(caption['tokens']) <= max_len:
            captions.append(caption['tokens'])

    # If captions is empty, what should we do here?
    if len(captions) == 0:
        continue

    img_path = os.path.join(img_obj['filepath'], img_obj['filename'])

    # What if this image cannot be found?
    if not os.path.exists(img_path): continue

    # Append the pair to the list
    if img_obj['split'] == 'train':
      train_img_cap_pairs.append([img_path, captions])
    elif img_obj['split'] == 'val':
      val_img_cap_pairs.append([img_path, captions])

## Data tranformation *

In [ ]:
# HDF5: HDF5 is a unique technology suite that makes possible the management
# of extremely large and complex data collections.

# 1. We will create 2 hdf5 files: 
#      train_images.hdf5, val_images.hdf5
# 2. We will create 5 json files: 
#      word_map.json -- contains a (word -> number) hash object
#      train_captions.json -- contains a list of encoded training captions
#      val_captions.json -- contains a list of encoded validation captions
#      train_caption_length.json -- contains a list of training caption lengths
#      val_caption_length.json -- contains a list of validation caption lengths

In [ ]:
# Word Encoding
# word_map: word    -> number (starting from 1)
#           <pad>   -> 0
#           <start> -> the second highest number
#           <end>   -> the highest number
word_map = {k: idx + 1 for idx, k in enumerate(word_set)}
word_map['<start>'] = len(word_map) + 1
word_map['<end>'] = len(word_map) + 1
word_map['<pad>'] = 0

# Save word map to a JSON
with open(os.path.join('word_map.json'), 'w') as j:
  json.dump(word_map, j)

In [ ]:
for img_cap_pairs, split in [[train_img_cap_pairs,'train'], [val_img_cap_pairs, 'val']]:
    # Save encoded captions and their lengths to JSON files
    h5py_path = os.path.join(split + '_images.hdf5')
    
    # remove it if the path exists
    if os.path.exists(h5py_path): os.remove(h5py_path)

    with h5py.File(h5py_path, 'a') as h:
        # Make a note of the number of captions we are sampling per image
        h.attrs['captions_per_image'] = captions_per_image

        # Create dataset inside HDF5 file to store images
        # we do channel first for the image
        images = h.create_dataset('images', (len(img_cap_pairs), 3, 256, 256), dtype='uint8')

        enc_captions = []
        caplens = []
        for index, img_cap_pair in enumerate(img_cap_pairs):
            img_path, captions = img_cap_pair

            if len(captions) < captions_per_image:
                # add some captions by randomly sampling from captions
                captions = captions + [choice(captions) for _ in range(captions_per_image - len(captions))]
            else:
                # randomly sample k from captions
                captions = sample(captions, captions_per_image)

            # Sanity check
            assert len(captions) == captions_per_image

            # Read image and transform it into (3, 256, 256)
            # Hint: use cv2, you will need to read, resize and transpose
            img = cv2.imread(img_path)
            img = cv2.resize(img, (256, 256))
            img = img.transpose(2, 0, 1)

            assert img.shape == (3, 256, 256)

            # Save image to HDF5 file
            images[index] = img
            for idx, caption in enumerate(captions):
                # Encode captions
                #   a list of numbers
                #   Format should be <start> word1 word2 ... wordN <end> <pad> <pad>...
                #   The total length should be equal to max_len
                enc_c = [word_map['<start>']] + [word_map[word] for word in caption] + \
                 [word_map['<end>']] + [word_map['<pad>']] * (max_len - len(caption))

                enc_captions.append(enc_c)
                caplens.append(len(caption) + 2)                

    with open(os.path.join(split + '_captions.json'), 'w') as j:
        json.dump(enc_captions, j)

    with open(os.path.join(split + '_caption_length.json'), 'w') as j:
        json.dump(caplens, j)

# Sanity check
print('caption length:', caplens[-1])
print('caption:', caption)
print('caption encoding:', enc_c)

#Helper functions *

In [ ]:
def save_checkpoint(data_name, epoch, epochs_since_improvement, encoder, decoder, encoder_optimizer, decoder_optimizer,
                    bleu4, is_best):
    state = {'epoch': epoch,
             'epochs_since_improvement': epochs_since_improvement,
             'bleu-4': bleu4,
             'encoder': encoder,
             'decoder': decoder,
             'encoder_optimizer': encoder_optimizer,
             'decoder_optimizer': decoder_optimizer}
    filename = 'checkpoint_' + data_name + '.pth.tar'
    torch.save(state, filename)
    # If this checkpoint is the best so far, store a copy so it doesn't get overwritten by a worse checkpoint
    if is_best:
        torch.save(state, 'BEST_' + filename)


class AverageMeter(object):
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

def accuracy(scores, targets, k):
    batch_size = targets.size(0)
    _, ind = scores.topk(k, 1, True, True)
    correct = ind.eq(targets.view(-1, 1).expand_as(ind))
    correct_total = correct.view(-1).float().sum()  # 0D tensor
    return correct_total.item() * (100.0 / batch_size)

# Model Architecture

## Encoder

In [ ]:
import torch
from torch import nn
import torchvision

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


class Encoder(nn.Module):
    def __init__(self, encoded_image_size=14):
        super(Encoder, self).__init__()
        self.enc_image_size = encoded_image_size

        resnet = torchvision.models.resnet101(pretrained=True)   # pretrained ImageNet ResNet-101

        # Remove linear and pool layers. Do you know why? (linear: we don't need to classify, we just want encoder pool: we want to change image size)
        modules = nn.Sequential(*(list(resnet.children())[:-2]))
        self.resnet = modules

        # Resize image to fixed size to allow input images of variable size
        self.adaptive_pool = nn.AdaptiveAvgPool2d(self.enc_image_size)

        # We want to avoid training encoder
        for param in resnet.parameters():
          param.requires_grad = False

    def forward(self, images):
        # Use resnet, apply adaptive_pool
        out = self.resnet(images)  # (batch_size, 2048, image_size/32, image_size/32)
        out = self.adaptive_pool(out)  # (batch_size, 2048, encoded_image_size, encoded_image_size)
        out = out.permute(0,2,3,1)  # (batch_size, encoded_image_size, encoded_image_size, 2048)
        return out



##Attention *

In [ ]:
class Attention(nn.Module):
    """
    Attention Network.
    """

    def __init__(self, encoder_dim, decoder_dim, attention_dim):
        """
        :param encoder_dim: feature size of encoded images
        :param decoder_dim: size of decoder's RNN
        :param attention_dim: size of the attention network
        """
        super(Attention, self).__init__()
        self.encoder_att = nn.Linear(encoder_dim, attention_dim)  # linear layer to transform encoded image
        self.decoder_att = nn.Linear(decoder_dim, attention_dim)  # linear layer to transform decoder's output
        self.full_att = nn.Linear(attention_dim, 1)  # linear layer to calculate values to be softmax-ed
        self.relu = nn.ReLU()
        self.softmax = nn.Softmax(dim=1)  # softmax layer to calculate weights

    def forward(self, encoder_out, decoder_hidden):
        """
        Forward propagation.
        :param encoder_out: encoded images, a tensor of dimension (batch_size, num_pixels, encoder_dim)
        :param decoder_hidden: previous decoder output, a tensor of dimension (batch_size, decoder_dim)
        :return: attention weighted encoding, weights
        """
        att1 = self.encoder_att(encoder_out)  # (batch_size, num_pixels, attention_dim)
        att2 = self.decoder_att(decoder_hidden)  # (batch_size, attention_dim)
        att = self.full_att(self.relu(att1 + att2.unsqueeze(1))).squeeze(2)  # (batch_size, num_pixels)
        alpha = self.softmax(att)  # (batch_size, num_pixels)
        attention_weighted_encoding = (encoder_out * alpha.unsqueeze(2)).sum(dim=1)  # (batch_size, encoder_dim)

        return attention_weighted_encoding, alpha


##Decoder *

In [ ]:

class DecoderWithAttention(nn.Module):
    """
    Decoder.
    """

    def __init__(self, attention_dim, embed_dim, decoder_dim, vocab_size, encoder_dim=2048, dropout=0.5):
        """
        :param attention_dim: size of attention network
        :param embed_dim: embedding size
        :param decoder_dim: size of decoder's RNN
        :param vocab_size: size of vocabulary
        :param encoder_dim: feature size of encoded images
        :param dropout: dropout
        """
        super(DecoderWithAttention, self).__init__()

        self.encoder_dim = encoder_dim
        self.attention_dim = attention_dim
        self.embed_dim = embed_dim
        self.decoder_dim = decoder_dim
        self.vocab_size = vocab_size
        self.dropout = dropout

        self.attention = Attention(encoder_dim, decoder_dim, attention_dim)  # attention network

        self.embedding = nn.Embedding(vocab_size, embed_dim)  # embedding layer
        self.dropout = nn.Dropout(p=self.dropout)
        self.decode_step = nn.LSTMCell(embed_dim + encoder_dim, decoder_dim, bias=True)  # decoding LSTMCell
        self.init_h = nn.Linear(encoder_dim, decoder_dim)  # linear layer to find initial hidden state of LSTMCell
        self.init_c = nn.Linear(encoder_dim, decoder_dim)  # linear layer to find initial cell state of LSTMCell
        self.f_beta = nn.Linear(decoder_dim, encoder_dim)  # linear layer to create a sigmoid-activated gate
        self.sigmoid = nn.Sigmoid()
        self.fc = nn.Linear(decoder_dim, vocab_size)  # linear layer to find scores over vocabulary
        self.init_weights()  # initialize some layers with the uniform distribution

    def init_weights(self):
        """
        Initializes some parameters with values from the uniform distribution, for easier convergence.
        """
        self.embedding.weight.data.uniform_(-0.1, 0.1)
        self.fc.bias.data.fill_(0)
        self.fc.weight.data.uniform_(-0.1, 0.1)

    def init_hidden_state(self, encoder_out):
        """
        Creates the initial hidden and cell states for the decoder's LSTM based on the encoded images.
        :param encoder_out: encoded images, a tensor of dimension (batch_size, num_pixels, encoder_dim)
        :return: hidden state, cell state
        """
        mean_encoder_out = encoder_out.mean(dim=1)
        h = self.init_h(mean_encoder_out)  # (batch_size, decoder_dim)
        c = self.init_c(mean_encoder_out)
        return h, c

    def forward(self, encoder_out, encoded_captions, caption_lengths):
        """
        Forward propagation.
        :param encoder_out: encoded images, a tensor of dimension (batch_size, enc_image_size, enc_image_size, encoder_dim)
        :param encoded_captions: encoded captions, a tensor of dimension (batch_size, max_caption_length)
        :param caption_lengths: caption lengths, a tensor of dimension (batch_size, 1)
        :return: scores for vocabulary, sorted encoded captions, decode lengths, weights, sort indices
        """

        batch_size = encoder_out.size(0)
        encoder_dim = encoder_out.size(-1)
        vocab_size = self.vocab_size

        # Flatten image
        encoder_out = encoder_out.view(batch_size, -1, encoder_dim)  # (batch_size, num_pixels, encoder_dim)
        num_pixels = encoder_out.size(1)

        # Sort input data by decreasing lengths; why? apparent below
        caption_lengths, sort_ind = caption_lengths.squeeze(1).sort(dim=0, descending=True)
        encoder_out = encoder_out[sort_ind]
        encoded_captions = encoded_captions[sort_ind]

        # Embedding
        embeddings = self.embedding(encoded_captions)  # (batch_size, max_caption_length, embed_dim)

        # Initialize LSTM state
        h, c = self.init_hidden_state(encoder_out)  # (batch_size, decoder_dim)

        # We won't decode at the <end> position, since we've finished generating as soon as we generate <end>
        # So, decoding lengths are actual lengths - 1
        decode_lengths = (caption_lengths - 1).tolist()

        # Create tensors to hold word predicion scores and alphas
        predictions = torch.zeros(batch_size, max(decode_lengths), vocab_size).to(device)
        alphas = torch.zeros(batch_size, max(decode_lengths), num_pixels).to(device)

        # At each time-step, decode by
        # attention-weighing the encoder's output based on the decoder's previous hidden state output
        # then generate a new word in the decoder with the previous word and the attention weighted encoding
        for t in range(max(decode_lengths)):
            batch_size_t = sum([l > t for l in decode_lengths])
            attention_weighted_encoding, alpha = self.attention(encoder_out[:batch_size_t],
                                                                h[:batch_size_t])
            gate = self.sigmoid(self.f_beta(h[:batch_size_t]))  # gating scalar, (batch_size_t, encoder_dim)
            attention_weighted_encoding = gate * attention_weighted_encoding
            h, c = self.decode_step(
                torch.cat([embeddings[:batch_size_t, t, :], attention_weighted_encoding], dim=1),
                (h[:batch_size_t], c[:batch_size_t]))  # (batch_size_t, decoder_dim)
            preds = self.fc(self.dropout(h))  # (batch_size_t, vocab_size)
            predictions[:batch_size_t, t, :] = preds
            alphas[:batch_size_t, t, :] = alpha

        return predictions, encoded_captions, decode_lengths, alphas, sort_ind

# Pytoch dataset transformation

In [ ]:
# h = h5py.File('train_images.hdf5', 'r')
# h['images'].shape[0]
# a = h.get('images')
# h['images']

In [ ]:
import torch
from torch.utils.data import Dataset
import h5py
import json
import os


class CaptionDataset(Dataset):
    def __init__(self, split):
        """
        :split, one of 'train', 'val'
        """
        self.split = split

        # Open hdf5 file where images are stored
        self.h = h5py.File(os.path.join(self.split + '_images.hdf5'), 'r')
        self.imgs = self.h.get('images')

        # Captions per image
        self.cpi = self.h.attrs['captions_per_image']

        # Load encoded captions 
        with open(os.path.join(self.split + '_captions.json'), 'r') as j:
          self.captions = json.load(j)

        # Load caption lengths
        with open(os.path.join(self.split + '_caption_length.json'), 'r') as j:
          self.caplens = json.load(j)
        
        # Total number of datapoints
        self.dataset_size = self.imgs.shape[0]

    def __getitem__(self, i):
        # Remember, the Nth caption corresponds to the (N // captions_per_image)th image
        img = torch.FloatTensor(self.imgs[i//self.cpi] / 255.)
        normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        img = transforms.Compose([normalize])(img)

        caption = torch.LongTensor(self.captions[i])
        caplen = torch.LongTensor([self.caplens[i]])

        if self.split is 'train':
            return img, caption, caplen
        else:
            # For validation of testing, also return all 'captions_per_image' captions to find BLEU-4 score
            all_captions = torch.LongTensor(
                self.captions[((i // self.cpi) * self.cpi):(((i // self.cpi) * self.cpi) + self.cpi)])
            return img, caption, caplen, all_captions

    def __len__(self):
        return self.dataset_size

# Model training

##Initialize parameters

In [ ]:
import time
import torch.backends.cudnn as cudnn
import torch.optim
import torch.utils.data
import torchvision.transforms as transforms
from torch import nn
from torch.nn.utils.rnn import pack_padded_sequence
from nltk.translate.bleu_score import corpus_bleu

# Model hyper-parameters
emb_dim = 512  # dimension of word embeddings
attention_dim = 512  # dimension of attention linear layers
decoder_dim = 512  # dimension of decoder RNN
dropout = 0.5
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  
cudnn.benchmark = True  

epochs = 30  
batch_size = 32
decoder_lr = 4e-4  
alpha_c = 1.  # regularization parameter for 'doubly stochastic attention'
best_bleu4 = 0.  # BLEU-4 score right now

## Training per epoch

In [ ]:
def train(train_loader, encoder, decoder, loss_function, decoder_optimizer, epoch):
    """
    Performs one epoch's training.
    :param train_loader: DataLoader for training data
    :param encoder: encoder model
    :param decoder: decoder model
    :param loss_function: loss layer
    :param decoder_optimizer: optimizer to update decoder's weights
    :param epoch: epoch number
    """

    decoder.train()  # train mode (dropout and batchnorm is used)
    encoder.train()

    losses = AverageMeter()  # loss (per word decoded)
    top5accs = AverageMeter()  # top5 accuracy

    # Load by batches
    for i, (imgs, caps, caplens) in enumerate(train_loader):
        # Remember to use GPU
        imgs = imgs.to(device)
        caps = caps.to(device)
        caplens = caplens.to(device)

        # Encoding
        encoded_imgs = encoder.forward(imgs)
        # Decoding
        preds, caps_sorted, decode_lengths, alphas, sort_ind = decoder.forward(encoded_imgs,caps,caplens)

        # Since we decoded starting with <start>, the targets are all words after <start>, up to <end>
        targets = caps_sorted[:, 1:]

        # Remove timesteps that we didn't decode at, or are pads
        preds, _, _, _ = pack_padded_sequence(preds, decode_lengths, batch_first=True)
        targets, _, _, _ = pack_padded_sequence(targets, decode_lengths, batch_first=True)

        # Calculate loss
        loss = loss_function(preds, targets)

        # Add doubly stochastic attention regularization
        loss += alpha_c * ((1. - alphas.sum(dim=1)) ** 2).mean()

        # Back prop on decoder only
        loss.backward()
      
        # Update weights
        decoder_optimizer.step()
        decoder_optimizer.zero_grad()

        # Keep track of metrics
        losses.update(loss.item(), sum(decode_lengths))
        top5accs.update(accuracy(preds, targets, 5), sum(decode_lengths))

        # Print status
        if i % 100 == 0:
            print('Epoch: [{0}][{1}/{2}]\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                  'Top-5 Accuracy {top5.val:.3f} ({top5.avg:.3f})'.format(epoch, i, len(train_loader),
                                                                          loss=losses,
                                                                          top5=top5accs))


## Validation per epoch *

In [ ]:
def validate(val_loader, encoder, decoder, loss_function):
    """
    Performs one epoch's validation.
    :param val_loader: DataLoader for validation data.
    :param encoder: encoder model
    :param decoder: decoder model
    :param loss_function: loss layer
    :return: BLEU-4 score
    """
    # eval mode (no dropout or batchnorm)
    decoder.eval()  
    encoder.eval()

    losses = AverageMeter()
    top5accs = AverageMeter()

    references = list()  # references (true captions) for calculating BLEU-4 score
    hypotheses = list()  # hypotheses (predictions)

    with torch.no_grad():
        # Batches
        for i, (imgs, caps, caplens, allcaps) in enumerate(val_loader):
            imgs = imgs.to(device)
            caps = caps.to(device)
            caplens = caplens.to(device)

            # Forward prop.
            encoded_imgs = encoder(imgs)
            scores, caps_sorted, decode_lengths, alphas, sort_ind = decoder(encoded_imgs, caps, caplens)
            targets = caps_sorted[:, 1:]

            # Remove timesteps that we didn't decode at, or are pads
            # pack_padded_sequence is an easy trick to do this
            scores_copy = scores.clone()

            scores, _, _, _ = pack_padded_sequence(scores, decode_lengths, batch_first=True)
            targets, _, _, _ = pack_padded_sequence(targets, decode_lengths, batch_first=True)

            # Calculate loss
            loss = loss_function(scores, targets)

            # Add doubly stochastic attention regularization
            loss += alpha_c * ((1. - alphas.sum(dim=1)) ** 2).mean()

            # Keep track of metrics
            losses.update(loss.item(), sum(decode_lengths))
            top5 = accuracy(scores, targets, 5)
            top5accs.update(top5, sum(decode_lengths))

            if i % 100 == 0:
                print('Validation: [{0}/{1}]\t'
                      'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                      'Top-5 Accuracy {top5.val:.3f} ({top5.avg:.3f})\t'.format(i, len(val_loader),
                                                                                loss=losses, top5=top5accs))

            # Store references (true captions), and hypothesis (prediction) for each image
            # If for n images, we have n hypotheses, and references a, b, c... for each image, we need -
            # references = [[ref1a, ref1b, ref1c], [ref2a, ref2b], ...], hypotheses = [hyp1, hyp2, ...]

            # References
            allcaps = allcaps[sort_ind]  # because images were sorted in the decoder
            for j in range(allcaps.shape[0]):
                img_caps = allcaps[j].tolist()
                img_captions = list(
                    map(lambda c: [w for w in c if w not in {word_map['<start>'], word_map['<pad>']}],
                        img_caps))  # remove <start> and pads
                references.append(img_captions)

            # Hypotheses
            _, preds = torch.max(scores_copy, dim=2)
            preds = preds.tolist()
            temp_preds = list()
            for j, p in enumerate(preds):
                temp_preds.append(preds[j][:decode_lengths[j]])  # remove pads
            preds = temp_preds
            hypotheses.extend(preds)

            assert len(references) == len(hypotheses)

        # Calculate BLEU-4 scores
        bleu4 = corpus_bleu(references, hypotheses)

        print(
            '\n * LOSS - {loss.avg:.3f}, TOP-5 ACCURACY - {top5.avg:.3f}, BLEU-4 - {bleu}\n'.format(
                loss=losses,
                top5=top5accs,
                bleu=bleu4))

    return bleu4


## Start training

In [ ]:
# Read word map
word_map_file = 'word_map.json'
with open(word_map_file, 'r') as j:
    word_map = json.load(j)

# Initialize encoder, and we don't train it
encoder = Encoder()
encoder_optimizer = None

# Initialize decoder, and adam optimizer
decoder = DecoderWithAttention(attention_dim, emb_dim, decoder_dim, len(word_map))
decoder_optimizer = torch.optim.Adam(
    params=filter(lambda p: p.requires_grad, decoder.parameters()),
    lr=decoder_lr
)


# Move to GPU, if available
decoder = decoder.to(device)
encoder = encoder.to(device)

# Loss function
loss_function = nn.CrossEntropyLoss().to(device)

# Custom dataloaders
train_loader = torch.utils.data.DataLoader(CaptionDataset('train'), batch_size=batch_size, shuffle=True)
val_loader = torch.utils.data.DataLoader(CaptionDataset('val'), batch_size=batch_size, shuffle=True)

# Epochs
for epoch in range(epochs):
    # One epoch's training
    train(train_loader, encoder, decoder, loss_function, decoder_optimizer, epoch)

    # One epoch's validation
    recent_bleu4 = validate(val_loader=val_loader,
                            encoder=encoder,
                            decoder=decoder,
                            loss_function=loss_function)

    # Save checkpoint
    is_best = recent_bleu4 > best_bleu4
    best_bleu4 = max(recent_bleu4, best_bleu4)

    save_checkpoint('coco', epoch, 0, encoder, decoder, encoder_optimizer,
                    decoder_optimizer, recent_bleu4, is_best)


Epoch: [0][0/26]	Loss 11.1749 (11.1749)	Top-5 Accuracy 0.000 (0.000)
Validation: [0/26]	Loss 6.4707 (6.4707)	Top-5 Accuracy 38.440 (38.440)	


/usr/local/lib/python3.6/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 3-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)



 * LOSS - 6.670, TOP-5 ACCURACY - 34.740, BLEU-4 - 0.11608515502843306

Epoch: [1][0/26]	Loss 6.9640 (6.9640)	Top-5 Accuracy 35.440 (35.440)
Validation: [0/26]	Loss 5.9584 (5.9584)	Top-5 Accuracy 39.481 (39.481)	

 * LOSS - 6.013, TOP-5 ACCURACY - 37.653, BLEU-4 - 0.27738807069616994

Epoch: [2][0/26]	Loss 6.0708 (6.0708)	Top-5 Accuracy 34.286 (34.286)
Validation: [0/26]	Loss 5.6096 (5.6096)	Top-5 Accuracy 41.551 (41.551)	

 * LOSS - 5.787, TOP-5 ACCURACY - 38.754, BLEU-4 - 0.29736372450441007

Epoch: [3][0/26]	Loss 5.9926 (5.9926)	Top-5 Accuracy 35.429 (35.429)
Validation: [0/26]	Loss 5.5444 (5.5444)	Top-5 Accuracy 39.892 (39.892)	

 * LOSS - 5.569, TOP-5 ACCURACY - 39.780, BLEU-4 - 0.01818392190427821

Epoch: [4][0/26]	Loss 6.0335 (6.0335)	Top-5 Accuracy 36.188 (36.188)
Validation: [0/26]	Loss 5.5172 (5.5172)	Top-5 Accuracy 41.111 (41.111)	

 * LOSS - 5.382, TOP-5 ACCURACY - 42.027, BLEU-4 - 0.0176615378819742

Epoch: [5][0/26]	Loss 5.5514 (5.5514)	Top-5 Accuracy 41.644 (41.644)
Val